In [1]:
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import os

import IPython.display as ipd

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,precision_recall_curve
from sklearn.metrics import precision_recall_fscore_support,f1_score,fbeta_score
from sklearn.metrics import make_scorer
from sklearn.metrics import classification_report
from sklearn.svm import SVC, NuSVC, LinearSVC
from sklearn.utils import shuffle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

import librosa
import librosa.display
import ast

import keras
plt.rcParams['figure.figsize'] = (17, 5)



Using TensorFlow backend.


In [2]:
df = pd.read_csv('data.csv')

In [3]:
df.head()

,filename,chroma_cqt,chroma_cens,chroma_stft,spectral_centroid,spectral_bandwidth,spectral_contrast,rolloff,zero_crossing_rate,tonnetz,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,126014,0.267008,0.171130,0.237383,699.229710,785.999971,25.676751,1183.751175,0.041712,0.092841,...,-27.326481,-20.607473,-12.659972,-13.275584,-12.988043,-15.687397,-12.339302,2.211766,2.147864,-5.011512
1,54496,0.274600,0.207362,0.277710,1833.074733,2091.124342,25.274995,3818.560243,0.093890,0.044640,...,-5.424025,6.843907,-6.247772,3.441898,-0.244973,4.090439,-1.278771,9.071951,-0.169724,0.161876
2,65893,0.458305,0.253559,0.412075,2767.487106,2741.112704,20.450162,5889.689386,0.111506,0.013648,...,-8.586940,6.649025,-2.908488,3.085722,-3.682684,2.957125,-2.243021,1.578797,-4.537785,1.052801
3,71158,0.531325,0.258276,0.424391,1444.225428,2060.084462,19.861984,3089.631317,0.046734,0.000029,...,5.303405,-0.030636,-1.089045,2.822888,-2.630777,1.943587,-7.131413,1.516889,-0.094114,-0.776687
4,57821,0.412456,0.239312,0.335196,2253.305912,2412.719760,22.327731,4858.028957,0.099870,0.024816,...,-8.056653,-5.495337,-8.891809,-2.783748,-6.777088,-3.736772,-11.871335,-0.826213,-2.934075,6.011378


In [4]:
df.shape

(7997, 30)

In [5]:
genres = pd.read_csv('data/fma_metadata/genrelist.csv')

In [6]:
genres

,Unnamed: 0,genre,testtrainval
0,NaN,NaN,NaN
1,track_id,NaN,NaN
2,2,Hip-Hop,training
3,5,Hip-Hop,training
4,10,Pop,training
...,...,...,...
7997,154308,Hip-Hop,test
7998,154309,Hip-Hop,test
7999,154413,Pop,training
8000,154414,Pop,training


In [7]:
genres.rename({'Unnamed: 0':'track_id'},axis=1,inplace=True)

In [8]:
genres = genres.drop(index=[0,1])

In [9]:
genres.head()

,track_id,genre,testtrainval
2,2,Hip-Hop,training
3,5,Hip-Hop,training
4,10,Pop,training
5,140,Folk,training
6,141,Folk,training


In [10]:
genres['track_id'] = genres['track_id'].astype('int64')

In [11]:
df.rename({'filename':'track_id'},axis=1,inplace=True)

In [12]:
df['track_id']

0       126014
1        54496
2        65893
3        71158
4        57821
         ...  
7992     43695
7993     68410
7994     51291
7995     96902
7996     75441
Name: track_id, Length: 7997, dtype: int64

In [13]:
data=pd.merge(df,genres,on='track_id')

In [14]:
data['genre']

0        Instrumental
1        Experimental
2                Folk
3             Hip-Hop
4             Hip-Hop
            ...      
7992    International
7993     Experimental
7994              Pop
7995     Experimental
7996     Instrumental
Name: genre, Length: 7997, dtype: object

In [15]:
data.set_index('track_id',inplace=True)

In [16]:
data.head()

,chroma_cqt,chroma_cens,chroma_stft,spectral_centroid,spectral_bandwidth,spectral_contrast,rolloff,zero_crossing_rate,tonnetz,mfcc1,...,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,genre,testtrainval
track_id,,,,,,,,,,,,,,,,,,,,,
126014,0.267008,0.171130,0.237383,699.229710,785.999971,25.676751,1183.751175,0.041712,0.092841,-352.161438,...,-12.659972,-13.275584,-12.988043,-15.687397,-12.339302,2.211766,2.147864,-5.011512,Instrumental,training
54496,0.274600,0.207362,0.277710,1833.074733,2091.124342,25.274995,3818.560243,0.093890,0.044640,-290.169464,...,-6.247772,3.441898,-0.244973,4.090439,-1.278771,9.071951,-0.169724,0.161876,Experimental,training
65893,0.458305,0.253559,0.412075,2767.487106,2741.112704,20.450162,5889.689386,0.111506,0.013648,-80.027184,...,-2.908488,3.085722,-3.682684,2.957125,-2.243021,1.578797,-4.537785,1.052801,Folk,training
71158,0.531325,0.258276,0.424391,1444.225428,2060.084462,19.861984,3089.631317,0.046734,0.000029,-123.564980,...,-1.089045,2.822888,-2.630777,1.943587,-7.131413,1.516889,-0.094114,-0.776687,Hip-Hop,training
57821,0.412456,0.239312,0.335196,2253.305912,2412.719760,22.327731,4858.028957,0.099870,0.024816,-71.340874,...,-8.891809,-2.783748,-6.777088,-3.736772,-11.871335,-0.826213,-2.934075,6.011378,Hip-Hop,training


In [17]:
data['testtrainval'].value_counts()

training      6397
validation     800
test           800
Name: testtrainval, dtype: int64

In [18]:
y = data[['genre','testtrainval']]

In [19]:
y.drop('testtrainval',axis=1,inplace=True)

In [20]:
X = data.drop(['genre','testtrainval'],axis=1)

In [21]:
X2 = X.iloc[:,9:29]

In [22]:
X2

,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
track_id,,,,,,,,,,,,,,,,,,,,
126014,-352.161438,147.268738,-6.516946,14.641133,29.060827,-2.149615,-9.781580,-8.744802,-11.623240,-17.436735,-27.326481,-20.607473,-12.659972,-13.275584,-12.988043,-15.687397,-12.339302,2.211766,2.147864,-5.011512
54496,-290.169464,107.796921,-17.293709,29.786123,8.302071,11.566362,-11.648953,-6.970734,-3.238187,-2.197775,-5.424025,6.843907,-6.247772,3.441898,-0.244973,4.090439,-1.278771,9.071951,-0.169724,0.161876
65893,-80.027184,61.457649,18.617395,38.681793,3.637755,21.321770,8.109901,4.545240,-0.697991,7.539818,-8.586940,6.649025,-2.908488,3.085722,-3.682684,2.957125,-2.243021,1.578797,-4.537785,1.052801
71158,-123.564980,127.734001,22.012423,42.193047,28.423532,-0.134702,7.428549,6.176642,-11.435475,4.944324,5.303405,-0.030636,-1.089045,2.822888,-2.630777,1.943587,-7.131413,1.516889,-0.094114,-0.776687
57821,-71.340874,92.360405,-3.542409,-0.463810,-13.841660,9.869255,-6.058197,0.128541,-7.656012,0.641387,-8.056653,-5.495337,-8.891809,-2.783748,-6.777088,-3.736772,-11.871335,-0.826213,-2.934075,6.011378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43695,-43.920574,101.167107,-54.982464,20.285261,-10.458629,19.034140,-2.695947,5.690491,-0.599940,2.894833,9.848370,17.061972,2.227302,-2.029427,-8.724811,2.876313,-2.736481,-2.547942,-10.971077,-4.219173
68410,-16.044903,70.045433,8.357967,28.988680,-3.816420,-5.503837,-0.557270,-0.938137,-1.681377,-6.005097,-4.045043,0.901013,-8.597396,-5.432930,-12.551827,-5.137374,-5.069657,1.975084,-0.862782,-1.661779
51291,-129.250885,63.081413,-90.161919,-19.360296,-11.701410,-1.094478,-11.671681,-0.070076,1.406723,9.443295,0.234382,4.351338,-6.455239,-0.917210,-7.075034,-6.199870,-16.941978,-10.509199,-13.883102,-8.334950


In [23]:
X_train, X_test, y_train , y_test = train_test_split(X,y,stratify=y,test_size=0.15)

In [24]:
X_train.shape,X_test.shape

((6797, 29), (1200, 29))

In [25]:
X_train

,chroma_cqt,chroma_cens,chroma_stft,spectral_centroid,spectral_bandwidth,spectral_contrast,rolloff,zero_crossing_rate,tonnetz,mfcc1,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
track_id,,,,,,,,,,,,,,,,,,,,,
131773,0.372388,0.209253,0.328619,1400.392424,1826.969921,22.535515,2757.141661,0.059253,-0.016174,-38.346336,...,-1.975244,3.768617,-4.153757,5.541670,-3.770152,-0.834110,-3.160408,0.150830,-9.146327,-1.456481
120782,0.334008,0.225040,0.264973,2589.118191,2139.619711,21.921884,4632.021991,0.164121,0.054004,-33.519344,...,-11.404074,0.371624,-17.020290,-7.852069,-14.927024,-4.611187,-14.555181,-6.223438,-4.563812,3.774600
54554,0.249969,0.174932,0.349217,2018.774915,2485.071715,22.808994,4536.830900,0.068653,-0.055521,-27.665665,...,-13.281664,-0.835726,-5.331334,0.038501,-10.288385,-2.898703,-7.998680,-6.829758,-12.564663,-6.673479
116487,0.382595,0.235406,0.300009,917.617663,1365.839534,24.056569,1742.831128,0.040990,0.019829,-280.326141,...,2.095576,-2.634427,3.689865,10.079310,-0.566289,-0.845538,-1.196876,-0.541258,3.024660,0.074682
124917,0.463618,0.255246,0.530601,2175.153524,2784.842095,19.442464,5311.234550,0.051251,0.002080,-120.701767,...,4.233308,10.707227,2.800570,6.826595,2.312478,5.476278,3.750225,8.809620,3.949850,3.140118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75754,0.605749,0.272646,0.440443,2159.743330,2425.641689,19.232929,4655.755181,0.093012,0.011497,-16.501081,...,0.357127,6.827625,-5.474258,6.541445,-4.097631,2.612027,-3.068569,-0.637887,-6.145115,-2.312634
36371,0.626633,0.275995,0.418540,2204.398216,2534.307242,18.307112,4803.279289,0.079183,0.014199,-28.272703,...,-2.384476,4.460902,-6.927068,3.699929,-2.150015,2.287759,-1.798179,3.438052,-2.824548,5.316834
21409,0.385574,0.230893,0.271134,1709.859690,2177.613298,21.935439,3631.719706,0.061976,-0.018494,-132.824417,...,-8.507709,-4.908473,-11.523382,-7.799408,-14.776809,-7.783975,-4.858551,2.836877,-3.028706,4.327224


In [26]:
1200/6397

0.18758793184305142

In [27]:
X_train,X_val, y_train, y_val = train_test_split(X_train,y_train, stratify=y_train,test_size=0.18)

In [99]:
encoder = LabelEncoder()

In [100]:
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)
y_val = encoder.transform(y_val)

In [101]:
scaler = StandardScaler()

In [102]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

In [102]:
X2_train = scaler.fit_transform(X2_train)
X2_test = scaler.transform(X2_test)
X2_val = scaler.transform(X2_val)

In [28]:
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=42)

In [29]:
classifier.score(X_train,y_train)

0.9899515521263234

In [30]:
classifier.score(X_test,y_test)

0.39416666666666667

In [31]:
classifier.score(X_val,y_val)

0.3872549019607843

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

In [33]:
 = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

In [112]:
max_features = ['auto', 'sqrt']


In [113]:
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

In [114]:
min_samples_split = [2, 5, 10]


In [115]:
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

In [116]:
random_grid = {'n_estimators': n_estimators,
               
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [32]:
rf_pipeline = Pipeline([('ss', StandardScaler()),('RF',RandomForestClassifier())])

In [43]:
params = [{'RF__max_depth': [4, 5, 6], 'RF__criterion':['gini','entropy'],
           'RF__max_features' :['auto', 'sqrt'],
         'RF__min_samples_split': [2, 5, 10], 
           'RF__n_estimators':[5,10,50,100,250,500],
         'RF__min_samples_leaf': [1, 2, 4]}]

In [44]:
rf_cv = GridSearchCV(estimator=rf_pipeline,param_grid=params,cv=5)

In [45]:
rf_cv.fit(X_val,y_val)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('ss', StandardScaler()),
                                       ('RF', RandomForestClassifier())]),
             param_grid=[{'RF__criterion': ['gini', 'entropy'],
                          'RF__max_depth': [4, 5, 6],
                          'RF__max_features': ['auto', 'sqrt'],
                          'RF__min_samples_leaf': [1, 2, 4],
                          'RF__min_samples_split': [2, 5, 10],
                          'RF__n_estimators': [100, 250, 500]}])

In [46]:
rf_best = rf_cv.best_estimator_
rf_cv.best_params_

{'RF__criterion': 'gini',
 'RF__max_depth': 6,
 'RF__max_features': 'sqrt',
 'RF__min_samples_leaf': 1,
 'RF__min_samples_split': 10,
 'RF__n_estimators': 500}

In [47]:
rf_best.score(X_train,y_train),rf_best.score(X_test,y_test)

(0.42490579580118426, 0.4)

In [49]:
rf_best.score(X_val,y_val)

0.6887254901960784

In [50]:
import xgboost as xgb


In [71]:
xgb_pipeline = Pipeline([('ss',StandardScaler()),('xgb',xgb.XGBClassifier(objective='multi',num_class=8))])


In [72]:
xgb_pipeline.fit(X_val,y_val)

Pipeline(steps=[('ss', StandardScaler()),
                ('xgb',
                 XGBClassifier(num_class=8, objective='multi:softprob'))])

In [81]:
xgb_pipeline.score(X_val,y_val)

0.8611111111111112

In [73]:
xgb_pipeline.score(X_train,y_train)

0.4211376278485555

In [74]:
xgb_pipeline.score(X_test,y_test)

0.4066666666666667

In [75]:
params = {
    "xgb__learning_rate": [0.001,0.05,0.01,0.1,0.5],
    'xgb__max_depth': [3,5,7],
    'xgb__subsample': [1],
    'xgb__n_estimators': [5, 50, 100, 250,500,750],
}

In [76]:
xgb_cv = GridSearchCV(estimator =xgb_pipeline,param_grid=params,n_jobs=-1,cv=5)

In [77]:
xgb_cv.fit(X_val,y_val)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('ss', StandardScaler()),
                                       ('xgb',
                                        XGBClassifier(num_class=8,
                                                      objective='multi:softprob'))]),
             n_jobs=-1,
             param_grid={'xgb__learning_rate': [0.001, 0.05, 0.01, 0.1, 0.5],
                         'xgb__max_depth': [3, 5, 7],
                         'xgb__n_estimators': [5, 50, 100, 250, 500, 750],
                         'xgb__subsample': [1]})

In [78]:
xgb_best = xgb_cv.best_estimator_
xgb_cv.best_params_

{'xgb__learning_rate': 0.01,
 'xgb__max_depth': 5,
 'xgb__n_estimators': 750,
 'xgb__subsample': 1}

In [79]:
xgb_best.score(X_train,y_train)

0.42687959806208503

In [80]:
xgb_best.score(X_test,y_test)

0.4

In [82]:
xgb_best.score(X_val,y_val)

0.9950980392156863